## Environment Setup

In [1]:
import os
from dotenv import load_dotenv

# Set environment variables in the .env file.
load_dotenv()

COMPLETION_MODEL = os.environ["OPENAI_COMPLETION_MODEL"]
COMPLETION_DEPLOYMENT = os.environ["OPENAI_COMPLETION_DEPLOYMENT"]
CHAT_MODEL = os.environ["OPENAI_CHAT_MODEL"]
CHAT_DEPLOYMENT = os.environ["OPENAI_CHAT_DEPLOYMENT"]
CHAT_API_VERSION = os.environ["OPENAI_CHAT_API_VERSION"]

MONGODB_CONNECTIONSTRING = os.environ["MONGODB_CONNECTIONSTRING"]
sessionId = 'test-session-id'

## Initiate MongoDB (using Azure CosmosDB)

In [3]:
from langchain.memory import MongoDBChatMessageHistory

history = MongoDBChatMessageHistory(
    connection_string=MONGODB_CONNECTIONSTRING,
    session_id=sessionId
)

history.clear()
history.add_ai_message("Hello, I am an AI assistant.")
history.add_user_message("Hello, I'm Kazuya.")
history.add_ai_message("Hi Kazuya, how can I help you today?")

In [5]:
# Test memory storage, try retrieving the history from Redis.
retrievedHistory = MongoDBChatMessageHistory(
    connection_string=MONGODB_CONNECTIONSTRING,
    session_id=sessionId
)

retrievedHistory.messages

[AIMessage(content='Hello, I am an AI assistant.', additional_kwargs={}, example=False),
 HumanMessage(content="Hello, I'm Kazuya.", additional_kwargs={}, example=False),
 AIMessage(content='Hi Kazuya, how can I help you today?', additional_kwargs={}, example=False)]

## LangChain MongoDB Memory

In [6]:
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import ConversationChain

llm = AzureOpenAI(
    model_name=COMPLETION_MODEL,
    deployment_name=COMPLETION_DEPLOYMENT,
    max_tokens=50,
    temperature=0.3,
    verbose=True
)

chat = AzureChatOpenAI(
    deployment_name=CHAT_DEPLOYMENT,
    openai_api_version=CHAT_API_VERSION,
    max_tokens=100,
    temperature=0.3,
    verbose=True
)

conversation = ConversationChain(
    llm=chat,
    verbose=True)

In [7]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    k = 2,
    chat_memory=retrievedHistory
)

conversation.memory = memory
conversation.predict(input="Can you tell me about our health benefits?")
conversation.predict(input="Can I claim for a new pair of glasses?")
conversation.predict(input="How about skin moisturizers?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
AI: Hello, I am an AI assistant.
Human: Hello, I'm Kazuya.
AI: Hi Kazuya, how can I help you today?
Human: Can you tell me about our health benefits?
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello, I'm Kazuya.
AI: Hi Kazuya, how can I help you today?
Human: Can you tell me about our health benefits?
AI: Sure, Kazuya. Our health benefits include medical, dental, a

"I'm sorry, Kazuya, but skin moisturizers are not typically covered under our health benefits. However, we do offer a wellness program that includes resources for maintaining healthy skin, such as tips for staying hydrated and protecting your skin from the sun. Would you like me to provide more information on our wellness program?"

## Check MongoDB

In [8]:
# Test memory storage, try retrieving the history from Redis.
latestHistory = MongoDBChatMessageHistory(
    connection_string=MONGODB_CONNECTIONSTRING,
    session_id=sessionId
)

latestHistory.messages

[AIMessage(content='Hello, I am an AI assistant.', additional_kwargs={}, example=False),
 HumanMessage(content="Hello, I'm Kazuya.", additional_kwargs={}, example=False),
 AIMessage(content='Hi Kazuya, how can I help you today?', additional_kwargs={}, example=False),
 HumanMessage(content='Can you tell me about our health benefits?', additional_kwargs={}, example=False),
 AIMessage(content='Sure, Kazuya. Our health benefits include medical, dental, and vision coverage. We also offer a wellness program that includes gym membership discounts, nutrition counseling, and mental health support. Our medical plan includes options for both HMO and PPO plans, with a range of deductibles and copays. Our dental plan covers preventative care, basic procedures, and major procedures. And our vision plan covers eye exams, glasses, and contact lenses. Is there anything specific you would like me to look up', additional_kwargs={}, example=False),
 HumanMessage(content='Can I claim for a new pair of glas